In [1]:
import tensorflow as tf
import os,datetime
import pandas as pd
import numpy as np
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import nltk
import re

2024-10-27 11:00:01.153729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730044801.197984  124660 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730044801.210477  124660 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-27 11:00:01.249760: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("../r4.2/http.csv",nrows=100000)
df

,id,date,user,pc,url,content
0,{V1Y4-S2IR20QU-6154HFXJ},01/02/2010 06:55:16,LRR0148,PC-4275,http://msn.com/The_Human_Centipede_First_Seque...,remain representatives consensus concert altho...
1,{Q5R1-T3EF87UE-2395RWZS},01/02/2010 07:00:13,NGF0157,PC-6056,http://urbanspoon.com/Plunketts_Creek_Loyalsoc...,festival off northwards than congestion partne...
2,{X9O1-O0XW52VO-5806RPHG},01/02/2010 07:03:46,NGF0157,PC-6056,http://aa.com/Rhodocene/rhodocenium/fhaavatqrf...,long away reorganized baldwin seth business 18...
3,{G5S8-U5OG04TE-5299CCTU},01/02/2010 07:05:26,IRM0931,PC-7188,http://groupon.com/Leonhard_Euler/leonhard/tne...,among german schwein experimental becomes prev...
4,{L0R4-A9DH29VP-4553AUWM},01/02/2010 07:05:52,IRM0931,PC-7188,http://flickr.com/Inauguration_of_Barack_Obama...,kate criteria j 2008 highest 12 include books ...
...,...,...,...,...,...,...
99995,{F2B4-J5HG21CP-2472KNWD},01/05/2010 08:24:11,JIM0095,PC-9328,http://city-data.com/No_Way_Out_2004/hotty/sre...,enlarged under generic advantage vision do any...
99996,{P4C0-H4AC44QZ-5898ATQE},01/05/2010 08:24:12,BBS0039,PC-9436,http://stubhub.com/Hoover_Dam/ickes/zbgbeplpyr...,an unknown afternoon dietary state law nationa...
99997,{U9Y2-H0WW90NF-1184ROHW},01/05/2010 08:24:12,KAL0395,PC-0004,http://tigerdirect.com/European_Commission/bar...,begin top we band themselves harshly or fourth...
99998,{Y2Y4-T1KW57MX-2017DWRE},01/05/2010 08:24:12,LDB0090,PC-6824,http://microsoft.com/Meteorological_history_of...,until acquire flared get secondary minas sea u...


In [3]:
# Download nltk data files (if not already installed)
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /homes/01/bxbhusal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /homes/01/bxbhusal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [5]:
# Example function to preprocess a single document
def preprocess(text):
    # Lowercase and remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    tokens = text.split()
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

In [6]:
preprocessed_content = [preprocess(doc) for doc in df['content']]

Step 2: Convert Documents to a Format Usable by LDA

LDA models typically work with bag-of-words (BoW) or term-frequency-inverse-document-frequency (TF-IDF) representations. You can use gensim to perform both vectorization and LDA modeling.

In [7]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.models import TfidfModel

# Create a dictionary and corpus for the LDA model
dictionary = Dictionary(preprocessed_content)

# we could olse trim low- and high-frequency words
# Here, we're saying filter out words which occur in fewer than N documents and more than M% of the documents

print("Length of the Dictionary before trimming:", len(dictionary))
dictionary.filter_extremes(no_below=5, no_above=0.5)
print("Length of the Dictionary after trimming:", len(dictionary))

corpus = [dictionary.doc2bow(doc) for doc in preprocessed_content]


# Optional: Convert to TF-IDF format
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

Length of the Dictionary before trimming: 45466
Length of the Dictionary after trimming: 28705


In [8]:
# Define the number of topics
num_topics = 10  # This should be tuned based on your data and requirements

# Train the LDA model
lda_model = LdaModel(corpus_tfidf, num_topics=num_topics, id2word=dictionary, passes=10)

In [9]:
# Function to get topic distribution for a document
def get_topic_vector(doc_bow):
    topic_distribution = lda_model.get_document_topics(doc_bow, minimum_probability=0)
    # Convert to a fixed-length vector for each document
    topic_vector = [prob for _, prob in topic_distribution]
    return topic_vector

# Apply to all documents
content_topic_vectors = [get_topic_vector(doc) for doc in corpus_tfidf]


Evaluation

Evaluating an LDA (Latent Dirichlet Allocation) model is an important step in topic modeling to determine the effectiveness of the model in discovering underlying topics in a corpus of text documents. There are several methods for evaluating an LDA model, including:

    Perplexity: Perplexity is a commonly used metric for evaluating LDA models. It measures how well the model predicts new data. A lower perplexity score indicates better performance. You can calculate perplexity using the log_perplexity() method of the LDA model in gensim.

    Coherence: Coherence measures the degree of semantic similarity between the top N words in each topic. Higher coherence scores indicate better topic quality. You can calculate coherence using the CoherenceModel class in gensim.

    Visualization: Visualization can be used to visually inspect the quality of the topics generated by the LDA model. You can use the pyLDAvis package in Python to create interactive visualizations of the topics.

    Human evaluation: Finally, you can also perform human evaluation by having human annotators review the topics generated by the LDA model and provide feedback on the quality and interpretability of the topics.

In summary, evaluating an LDA model involves using one or more of the above methods to measure the quality of the topics generated by the model. By evaluating the model, you can make adjustments to the parameters and improve the quality of the topics.


In [13]:
lda_model.log_perplexity(corpus)

-9.338984215079103

In [14]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=preprocessed_content,
                                     dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score:', coherence_lda) 

Coherence Score: 0.3823179442671656



Hyperparameter Tuning¶

First, let’s differentiate between model hyperparameters and model parameters :

    Number of Topics (K)
    Dirichlet hyperparameter alpha: Document-Topic Density
    Dirichlet hyperparameter beta: Word-Topic Density



In [22]:
from gensim import models

# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = models.LdaModel(corpus=corpus, num_topics=k,
                                id2word=dictionary, passes=10,
                                alpha=a, eta=b, chunksize=100,
                                per_word_topics=True, random_state=42)
    coherence_model_lda = CoherenceModel(model=lda_model, 
                                         texts=preprocessed_content,
                                         dictionary=dictionary, coherence='c_v')
    return coherence_model_lda.get_coherence()

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import sys
from gensim.utils import ClippedCorpus

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 10
max_topics = 30
step_size = 5
topics_range = range(min_topics, max_topics+1, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')


# Validation sets
num_of_docs = len(corpus)
corpus_sets = [ClippedCorpus(corpus, int(num_of_docs*0.75)), corpus]
corpus_title = ['75% Corpus', '100% Corpus']

# results placeholder
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# tqdm progress bar
pbar = tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)),
                 file=sys.stdout, colour='green')

### takes ~ 37 minutes
for i in range(len(corpus_sets)):
    for k in topics_range:    # iterate through validation corpuses
        for a in alpha:       # iterate through alpha values
            for b in beta:    # iterare through beta values
                # get the coherence score for the given parameters
                cv = compute_coherence_values(corpus=corpus_sets[i],
                                              dictionary=dictionary,
                                              k=k, a=a, b=b)
                # Save the model results
                model_results['Validation_Set'].append(corpus_title[i])
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)
                
                # update tqdm progress bar
                pbar.update(1)
                pbar.refresh()
pbar.close()

# save the results to a dataframe
res_df = pd.DataFrame(model_results)

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
res_df.sort_values(by='Coherence', ascending=False).head()

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

res_df.Coherence.plot(kind='line', title='Coherence Score');

In [ ]:
res_df.Coherence.plot(kind='hist', title='Coherence Score Distribution');

In [ ]:
res_df.plot.scatter(x='Topics', y='Coherence', title='Topics vs Coherence');

In [ ]:
res_df.groupby('Topics')['Coherence'].mean().plot(title='Topics Coherence Mean');